In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

## Library

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor,LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl

import kaggle_evaluation.mcts_inference_server

In [3]:
train_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"
test_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv"

In [4]:
train_data=pd.read_csv(train_path)
test_data =pd.read_csv(test_path)

In [5]:
train_data 

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [6]:
train_data.describe()

,Id,Properties,Format,Time,Discrete,Realtime,Turns,Alternating,Simultaneous,Stochastic,...,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
count,233234.000000,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.000000,...,233234.000000,233234.000000,233234.000000,233234.000000,233234.000000,2.332340e+05,233234.000000,233234.000000,233234.000000,233234.000000
mean,116616.500000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.075829,...,0.680055,0.650042,0.270158,0.018385,2863.386342,1.666635e+05,6.759735,2.333528,6.087586,0.044398
std,67329.000679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.264726,...,0.466456,0.476957,0.444042,0.134339,9558.007843,1.890782e+05,5.377815,4.625684,5.272134,0.622847
min,0.000000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.030000,5.690000e+00,0.000000,0.000000,0.000000,-1.000000
25%,58308.250000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,95.870000,4.047234e+04,1.000000,0.000000,1.000000,-0.333333
50%,116616.500000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,1.000000,1.000000,0.000000,0.000000,449.790000,1.054746e+05,6.000000,0.000000,6.000000,0.000000
75%,174924.750000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,1.000000,1.000000,1.000000,0.000000,1275.520000,2.379821e+05,11.000000,2.000000,10.000000,0.533333
max,233233.000000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,164877.370000,2.096745e+06,44.000000,30.000000,45.000000,1.000000


In [7]:
print(len(train_data["agent2"].unique()))

72


In [8]:
def summarize_dataframe_info(df):
    table=pd.DataFrame(columns=["Column","Data Type","Null count"])
    Dtype=""
    null=0
    for column in df.columns:
        Dtype=df[column].dtype
        null=df[column].isnull().sum()
        new_row = {"Column": column, "Data Type": Dtype,"Null count":null}
        table = pd.concat([table, pd.DataFrame([new_row])], ignore_index=True)
    return table
        
        

In [9]:
info=summarize_dataframe_info(train_data)

In [10]:
info[info["Null count"]>=1]

,Column,Data Type,Null count
550,Behaviour,float64,233234
551,StateRepetition,float64,233234
554,Duration,float64,233234
560,Complexity,float64,233234
564,BoardCoverage,float64,233234
568,GameOutcome,float64,233234
575,StateEvaluation,float64,233234
576,Clarity,float64,233234
579,Decisiveness,float64,233234
584,Drama,float64,233234


In [11]:
info[info["Data Type"]=="object"]

,Column,Data Type,Null count
1,GameRulesetName,object,0
2,agent1,object,0
3,agent2,object,0
808,EnglishRules,object,0
809,LudRules,object,0


In [12]:
train_data["GameRulesetName"].unique()

array(['00Y', '1D_Chess', '20_SquaresSeleucid_Scholarly', ...,
       'Zurgaan_Tal', 'Zuz_Mel_5x5', 'Zuz_Mel_7x7'], dtype=object)

In [13]:
train_data["agent1"].unique()

array(['MCTS-ProgressiveHistory-0.1-MAST-false',
       'MCTS-ProgressiveHistory-0.1-MAST-true',
       'MCTS-ProgressiveHistory-0.1-NST-false',
       'MCTS-ProgressiveHistory-0.1-NST-true',
       'MCTS-ProgressiveHistory-0.1-Random200-false',
       'MCTS-ProgressiveHistory-0.1-Random200-true',
       'MCTS-ProgressiveHistory-0.6-MAST-false',
       'MCTS-ProgressiveHistory-0.6-MAST-true',
       'MCTS-ProgressiveHistory-0.6-NST-false',
       'MCTS-ProgressiveHistory-0.6-NST-true',
       'MCTS-ProgressiveHistory-0.6-Random200-false',
       'MCTS-ProgressiveHistory-0.6-Random200-true',
       'MCTS-ProgressiveHistory-1.41421356237-MAST-false',
       'MCTS-ProgressiveHistory-1.41421356237-MAST-true',
       'MCTS-ProgressiveHistory-1.41421356237-NST-false',
       'MCTS-ProgressiveHistory-1.41421356237-NST-true',
       'MCTS-ProgressiveHistory-1.41421356237-Random200-false',
       'MCTS-ProgressiveHistory-1.41421356237-Random200-true',
       'MCTS-UCB1-0.1-MAST-false', 'MCTS-UC

Drop columns that have 95% or more identical values and columns with all null values.

In [14]:
def drop_column_name(df):
    drop_column=[]
    for column in df.columns:
        if df[column].value_counts().max()/233234 >= .95 or len(df[column].value_counts())==0:
            drop_column.append(column)
    return drop_column
drop_column=drop_column_name(train_data)
print (drop_column)

['Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating', 'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules', 'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire', 'TwoPlayer', 'Multiplayer', 'Cooperation', 'Team', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 'Container', 'Board', 'TriangleShape', 'DiamondShape', 'SpiralShape', 'CircleShape', 'PrismShape', 'StarShape', 'ParallelogramShape', 'SquarePyramidalShape', 'RectanglePyramidalShape', 'TargetShape', 'TriangleTiling', 'BrickTiling', 'SemiRegularTiling', 'CelticTiling', 'MorrisTiling', 'QuadHexTiling', 'CircleTiling', 'ConcentricTiling', 'SpiralTiling', 'AlquerqueTiling', 'MancalaStores', 'MancalaThreeRows', 'MancalaSixRows', 'MancalaCircular', 'AlquerqueBoardWithOneTriangle', 'AlquerqueBoardWithTwoTriangles', 'AlquerqueBoardWithFourTriangles', 'AlquerqueBoardWithEightTriangles', 'ThreeMensMorrisBoard', 'Thr

add some column we will not use

In [15]:
drop_column.extend(["Id","agent1","agent2","EnglishRules","LudRules","num_wins_agent1","num_draws_agent1","num_losses_agent1"])

In [16]:
print (len(drop_column)) 

485


In [17]:
train=train_data.drop(columns=drop_column, errors='ignore')


In [18]:
le = LabelEncoder()

train["GameRulesetName"] = le.fit_transform(train["GameRulesetName"])

In [19]:
test=test_data.drop(columns=drop_column, errors='ignore')
test["GameRulesetName"] = le.transform(test["GameRulesetName"])

In [20]:
train

,GameRulesetName,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Shape,SquareShape,HexShape,RectangleShape,...,SetPending,PlayerValue,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,PlayoutsPerSecond,MovesPerSecond,utility_agent1
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,298.07,18877.17,-0.466667
1,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,298.07,18877.17,-0.333333
2,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,298.07,18877.17,-0.066667
3,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,298.07,18877.17,-0.333333
4,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,298.07,18877.17,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,1376,0,0,0,0,0,1,1,0,0,...,0,0,1,0,1,1,0,157.52,157174.58,-0.733333
233230,1376,0,0,0,0,0,1,1,0,0,...,0,0,1,0,1,1,0,157.52,157174.58,0.266667
233231,1376,0,0,0,0,0,1,1,0,0,...,0,0,1,0,1,1,0,157.52,157174.58,0.666667
233232,1376,0,0,0,0,0,1,1,0,0,...,0,0,1,0,1,1,0,157.52,157174.58,0.666667


In [21]:
test

,GameRulesetName,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Shape,SquareShape,HexShape,RectangleShape,...,ForgetValues,SetPending,PlayerValue,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,PlayoutsPerSecond,MovesPerSecond
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,298.07,18877.17
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,298.07,18877.17
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,298.07,18877.17


In [22]:
X = train.drop('utility_agent1', axis=1)  # Features
y = train['utility_agent1'] 

In [23]:
X_train,X_valid,y_train,y_valid  =train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
model = CatBoostRegressor(iterations=1000, 
                          learning_rate=0.05, 
                          depth=6)



In [25]:

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=True)

0:	learn: 0.6148084	test: 0.6160817	best: 0.6160817 (0)	total: 166ms	remaining: 2m 45s
1:	learn: 0.6072438	test: 0.6083911	best: 0.6083911 (1)	total: 277ms	remaining: 2m 18s
2:	learn: 0.5996848	test: 0.6007673	best: 0.6007673 (2)	total: 447ms	remaining: 2m 28s
3:	learn: 0.5935408	test: 0.5946244	best: 0.5946244 (3)	total: 593ms	remaining: 2m 27s
4:	learn: 0.5874854	test: 0.5885771	best: 0.5885771 (4)	total: 739ms	remaining: 2m 27s
5:	learn: 0.5818998	test: 0.5829499	best: 0.5829499 (5)	total: 883ms	remaining: 2m 26s
6:	learn: 0.5767033	test: 0.5776678	best: 0.5776678 (6)	total: 1.03s	remaining: 2m 26s
7:	learn: 0.5717886	test: 0.5727370	best: 0.5727370 (7)	total: 1.21s	remaining: 2m 29s
8:	learn: 0.5672015	test: 0.5681954	best: 0.5681954 (8)	total: 1.36s	remaining: 2m 30s
9:	learn: 0.5631561	test: 0.5640793	best: 0.5640793 (9)	total: 1.5s	remaining: 2m 29s
10:	learn: 0.5591423	test: 0.5600375	best: 0.5600375 (10)	total: 1.66s	remaining: 2m 29s
11:	learn: 0.5557034	test: 0.5566019	best:

In [26]:
predictions = model.predict(test)

# Evaluate the model


In [27]:
predictions

array([-0.03872164, -0.03872164, -0.03872164])

In [28]:
submission = pd.DataFrame({
    'Id': test_data["Id"],
    'utility_agent1': predictions
})

In [29]:
submission

,Id,utility_agent1
0,233234,-0.038722
1,233235,-0.038722
2,233236,-0.038722


In [30]:
tables = pa.Table.from_pandas(submission)

# Write the PyArrow Table to a Parquet file
pq.write_table(tables, 'submission.parquet')

print("\nSubmission file created: submission.parquet")


Submission file created: submission.parquet


In [31]:
tables

pyarrow.Table
Id: int64
utility_agent1: double
----
Id: [[233234,233235,233236]]
utility_agent1: [[-0.03872163598185452,-0.03872163598185452,-0.03872163598185452]]